<a href="https://colab.research.google.com/github/mouslimdiallo/FacenetModel/blob/main/FaceNet_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import files
#files.upload()

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!git clone https://github.com/mouslimdiallo/FaceNet_Recognition.git
!git clone https://github.com/nyoki-mtl/keras-facenet.git

Cloning into 'FaceNet_Recognition'...
fatal: could not read Username for 'https://github.com': No such device or address
Cloning into 'keras-facenet'...
remote: Enumerating objects: 57, done.
remote: Total 57 (delta 0), reused 0 (delta 0), pack-reused 57
Unpacking objects: 100% (57/57), done.


In [2]:
!unzip /content/drive/MyDrive/Colab\ Notebooks/Reconnaissance\ Faciale\ Project/keras-facenet.zip


Archive:  /content/drive/MyDrive/Colab Notebooks/Reconnaissance Faciale Project/keras-facenet.zip
  inflating: keras-facenet/model/facenet_keras.h5  
  inflating: keras-facenet/weights/facenet_keras_weights.h5  


In [3]:
# import du modele FaceNet Keras
from keras.models import load_model
# Charger le modele
model = load_model('/content/keras-facenet/model/facenet_keras.h5')
# Resumer des input et output
print(model.inputs)
print(model.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


Là nous avons un modèle keras-facenet prêt à l'emploi avec des tenseurs d'entrée et de sorties qui fonctionnent avec des images pixelisées 160x160 px en entrée.

Installation de MTCNN, PIL, PLOT et Numpy

In [4]:
# installation et import des bibliothèques necessaires
!pip install mtcnn
!pip install praw
!pip install Pillow
!pip install numpy
!pip install matplotlib

     |████████████████████████████████| 2.3 MB 5.1 MB/s 
     |████████████████████████████████| 176 kB 5.2 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 


In [5]:
# Pour un dataset sur Kaggle, voici comment l'importer directement sur Colab.
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
# La commande pour telecharger notre dataset kaggle
!kaggle datasets download -d dansbecker/5-celebrity-faces-dataset

  0% 0.00/5.03M [00:00<?, ?B/s]
100% 5.03M/5.03M [00:00<00:00, 69.9MB/s]


In [ ]:
# dezipper les dataset kaggle des celebrités
!unzip /content/5-celebrity-faces-dataset.zip

Archive:  /content/5-celebrity-faces-dataset.zip
replace data/train/ben_afflek/httpcsvkmeuaeccjpg.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Import d'un dataset kaggle pour la classification d'images de celebrités : train et test/

On fait l'entrainemnt dans train, et nous classons dans validation. Exemple à reproduire avec d'autres dataset.

detectons chaque visage dans chaque photo (celebrité ) et faire une extraction.



L'objectif est de prédire l'identité d'un visage donné avec un classificateur SVM (Linear Support Vector Machine).

In [ ]:
# import des outils face detection avec mtcnn
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
import numpy as np


# Extraction d'un seul visage avec px 160
def extract_face(filename, required_size=(160, 160)):
  # charger l'image à partir du fichier
  image = Image.open(filename)
  # convertir en l'image en RGB
  image = image.convert('RGB')
  # convertir en tableau
  pixels = asarray(image)
  # creer notre detecteur de visage mtcnn
  detector = MTCNN()
  # detection du visage stocké dans results
  results = detector.detect_faces(pixels)
  # extraire le cadre de limitation de la première face
  x1, y1, width, height = results[0]['box']

  # corriger les erreurs
  x1, y1 = abs(x1), abs(y1)
  x2, y2 = x1 + width, y1 + height
  # extraire le visage
  face = pixels[y1:y2, x1:x2]
  # redimension des pixels du modele
  image = Image.fromarray(face)
  image = image.resize(required_size)
  face_array = asarray(image)
  return face_array

# spécificions le dossier dans lequel on va faire le traçage
folder = '/content/data/train/ben_afflek/'
i = 1
# Faire une liste d'énumeration des fichiers dans le dossier
for filename in listdir(folder):
# Le chemin d'accès
	path = folder + filename
	  # Obtention du visage
	face = extract_face(path)
	print(i, face.shape)
	# faire la tracée
	pyplot.subplot(2, 7, i)
	pyplot.axis('off')
	pyplot.imshow(face)
	i += 1
pyplot.show()

Chargement de chaque visage dans un tableau numpy

Voici le chargement de chaque photo sous forme de tableau numpy. Mainteannt parcourons chaque sous repertoire afin de detecter les visages pour coller des etiquettes dessus. Utilisons des fonctions pour cela.

In [ ]:
#import des outils necessaires
from os import listdir
from os.path import isdir
from PIL import Image
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# Ici on reprend la même procedure, puis...
# Extraction d'un seul visage avec px 160 en utilisant la fonction extract
def extract_face(filename, required_size=(160, 160)):
  # charger l'image à partir du fichier
  image = Image.open(filename)
  # convertir en l'image en RGB
  image = image.convert('RGB')
  # convertir en tableau
  pixels = asarray(image)
  # creer notre detecteur de visage mtcnn
  detector = MTCNN()
  # detection du visage stocké dans results
  results = detector.detect_faces(pixels)
  # extraire le cadre de limitation de la première face
  x1, y1, width, height = results[0]['box']

  # corriger les erreurs
  x1, y1 = abs(x1), abs(y1)
  x2, y2 = x1 + width, y1 + height
  # extraire le visage
  face = pixels[y1:y2, x1:x2]
  # redimension des pixels du modele
  image = Image.fromarray(face)
  image = image.resize(required_size)
  face_array = asarray(image)
  return face_array

# Utiliser la fonction loadfaces pour étiquetter tous les répertoires des celebrités
def load_faces(directory):
  faces = list()
  # enumerrer les fichiers
  for filename in listdir(directory):
    # Definir le chemin
    path = directory + filename
    # Obtenir le visage
    face = extract_face(path)
    # Tracer
    faces.append(face)
  return faces

# la fonction load_dataset : detection des visages dans les sous repertoires et l'attribution d'étiquettes
# charger un ensemble de données qui contient un sous-répertoire pour chaque classe qui à son tour contient des images
def load_dataset(directory):
  X, y = list(), list()
  # # enumerrer les fichiers par classe 
  for subdir in listdir(directory):
    # Definir le chemin
    path = directory + subdir + '/'
    # Ignorer tous les fichiers qui pourraient être dans le répertoire
    if not isdir(path):
		    continue
    # charge toutes les faces du sous-répertoire
    faces = load_faces(path)
    # creation des étiquettes
    labels = [subdir for _ in range(len(faces))]
    # Résumé de la progression
    print('> %d exemples chargés pour la classe : %s' % (len(faces), subdir))
    # Tracer
    X.extend(faces)
    y.extend(labels)
  return asarray(X), asarray(y)

# Charger notre dataset

trainX, trainy = load_dataset('data/train/')
print(trainX.shape, trainy.shape)
# Charger le dataset pour le test
testX, testy = load_dataset('/content/data/val')
print(testX.shape, testy.shape)
# enregistrer les tableaux dans un fichier au format compressé
savez_compressed('/content/data', trainX, trainy, testX, testy) 


Maintenant chargeons notre dataset npz de visages detectés et utilisons facenet pour l'integration de visage. L'objectif, prédire les visages, pour cela, traitons les pixels de images pour facenet.

In [ ]:

# calculer une intégration de visage pour chaque visage dans l'ensemble de données à l'aide de facenet
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model

# obtenir l'intégration (signification) du visage pour un visage
def get_embedding(model, face_pixels):
  # mettre à l'échelle les valeurs des pixels
  face_pixels = face_pixels.astype('float32')
  # standardiser les valeurs de pixels sur tous les canaux (global)
  mean, std = face_pixels.mean(), face_pixels.std()
  face_pixels = (face_pixels - mean) / std
  # transformer le visage en un seul échantillon
  samples = expand_dims(face_pixels, axis=0)
  # faire une prédiction pour obtenir l'intégration
  yhat = model.predict(samples)
  return yhat[0]


# Charger les visages dataset
data = load('/content/data.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Chargé : ', trainX.shape, trainy.shape, testX.shape, testy.shape)


# Charger le facenet model pour integratin de visage
model = load_model('/content/keras-facenet/model/facenet_keras.h5')
print('Notre modèle chargé')

# Enumerer chaque visage pour faire de l'integration
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

# convertir chaque visage de l'ensemble de test en une intégration
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

# enregistrer les tableaux dans un fichier au format compressé
savez_compressed('data_integration.npz', newTrainX, trainy, newTestX, testy)


L'ensemble est enregistré sous forme de tableau numpy dans un repertoire compressé.

Maintenant classons les inscrustations de visage à l'aide de notre modèle. Utilisation de en utilisant la classe Normalizer dans scikit-learn et la la classe LabelEncoder. Pour normaliser, les vecteurs utilisons SVC de SVM dans scikit-learn. voyons

La classification des visages

In [ ]:
# développer un classificateur pour l'ensemble de données 5 Celebrity Faces et charger les biblio
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC


# Charger dataset d'integration de visage
data = load('/content/data_integration.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))

# normaliser les vecteurs d'insertion de visage
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)

# encodage d'étiquettes pour les cibles
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)

# Entrainement et ajustement du modele
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)


# Notre prediction
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)

# Voir le score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# Notre summarize
print('Précision: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Pour plus de précision de notre résultat

In [ ]:
# Notre classificateur pour les celebrités
from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot

# Nous reprenons presque la même méthode

# Chargement de nos visages
data = load('5-celebrity-faces-dataset.npz')
testX_faces = data['arr_2']
# visage d'integration chargé
data = load('data_integration.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']

# normalisation des vecteurs
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)

# labelisation des cibles
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)

# ajustement du modele
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)

# modèle de test sur un exemple aléatoire de l'ensemble de données de test
selection = choice([i for i in range(testX.shape[0])])
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])

# prediction du visage 
samples = expand_dims(random_face_emb, axis=0)
yhat_class = model.predict(samples)
yhat_prob = model.predict_proba(samples)

# Obtenir l'étiquette
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)

print('Visage prédit : %s (%.3f)' % (predict_names[0], class_probability))
print('Visage attendue: %s' % random_face_name[0])

# Traçage
pyplot.imshow(random_face_pixels)
title = '%s (%.3f)' % (predict_names[0], class_probability)
pyplot.title(title)
pyplot.show()

Dans ce projet, nous avons utiliser un modèle Facenet et un classificateur SVM pour identifier les visages sur des images.

Nous avons préparés les datasets, effectuer une detection de visages en extrayant les caracteristiques de visages et l'integration de visages.
Puis, nous avons utiliser SVM pour l'identification dans un système d'integration de visages.

Resultat : nom d'etiquette et prédiction.

Nous allons aller en approfondir le sujet de l'apprentissage en profondeur pour l'ordinateur
vision.